In [ ]:
import torch
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import time
import copy
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler


def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                print("running_loss:{}, running_corrects:{}".format(running_loss, running_corrects))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


print("start")
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                              std=[0.229, 0.224, 0.225])
    ])
test_data_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])

artists_dataset = datasets.ImageFolder(root='/home/lab/yasuhiro/Downloads/best-artworks-of-all-time/images/images',
                                           transform=data_transform)
dataset_loader = torch.utils.data.DataLoader(artists_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=0)

test_dataset = datasets.ImageFolder(root='/home/lab/yasuhiro/Downloads/best-artworks-of-all-time/test',
                                           transform=test_data_transform)

test_dataset_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=0)

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/hymenoptera_data'
image_datasets = {'train':artists_dataset ,'val':test_dataset}
dataloaders = {'train':dataset_loader ,'val':test_dataset_loader}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print("Dataloader OK")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device OK")


# Get a batch of training data
print("start iter")
# inputs, classes = next(iter(dataset_loader))
print("make grid")
# Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[artists_dataset.classes[x] for x in classes])




print("start fit model")
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 51)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

print("model OK")

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2)

visualize_model(model_ft)



start
Dataloader OK
device OK
start iter
make grid
start fit model
model OK
Epoch 0/1
----------
running_loss:16.988262176513672, running_corrects:0
running_loss:33.38073539733887, running_corrects:0
running_loss:49.74155235290527, running_corrects:0
running_loss:66.99276924133301, running_corrects:0
running_loss:82.3464412689209, running_corrects:0
running_loss:97.25974082946777, running_corrects:0
running_loss:114.4439926147461, running_corrects:0
running_loss:130.38373184204102, running_corrects:0
running_loss:146.57840538024902, running_corrects:0
running_loss:161.50343322753906, running_corrects:0
running_loss:175.6491117477417, running_corrects:1
running_loss:193.7445993423462, running_corrects:1
running_loss:206.4851655960083, running_corrects:1
running_loss:221.49515628814697, running_corrects:1
running_loss:242.06612491607666, running_corrects:1
running_loss:256.71098804473877, running_corrects:1
running_loss:268.3757028579712, running_corrects:1
running_loss:282.5605735778808

running_loss:2275.0597581863403, running_corrects:69
running_loss:2288.4267625808716, running_corrects:70
running_loss:2298.9215812683105, running_corrects:71
running_loss:2316.8906860351562, running_corrects:71
running_loss:2330.233558654785, running_corrects:72
running_loss:2342.468448638916, running_corrects:74
running_loss:2360.1005477905273, running_corrects:75
running_loss:2372.688111305237, running_corrects:76
running_loss:2387.205620765686, running_corrects:76
running_loss:2400.8427953720093, running_corrects:76
running_loss:2413.338376045227, running_corrects:77
running_loss:2427.348346710205, running_corrects:78
running_loss:2440.2105407714844, running_corrects:78
running_loss:2452.2782917022705, running_corrects:79
running_loss:2462.298454284668, running_corrects:80
running_loss:2476.3053150177, running_corrects:80
running_loss:2487.5025005340576, running_corrects:82
running_loss:2501.2566328048706, running_corrects:82
running_loss:2512.2533712387085, running_corrects:84
run

running_loss:4274.104307174683, running_corrects:180
running_loss:4280.936020851135, running_corrects:181
running_loss:4294.079188346863, running_corrects:181
running_loss:4307.594882965088, running_corrects:181
running_loss:4320.364458084106, running_corrects:183
running_loss:4331.454313278198, running_corrects:184
running_loss:4337.870357036591, running_corrects:187
running_loss:4350.599445819855, running_corrects:187
running_loss:4365.358126163483, running_corrects:188
running_loss:4380.095009326935, running_corrects:188
running_loss:4391.108868122101, running_corrects:189
running_loss:4400.870898723602, running_corrects:191
running_loss:4416.279616832733, running_corrects:191
running_loss:4429.608314037323, running_corrects:192
running_loss:4446.405471324921, running_corrects:192
running_loss:4459.902371883392, running_corrects:193
running_loss:4476.513509273529, running_corrects:193
running_loss:4487.142205715179, running_corrects:195
running_loss:4500.724359035492, running_correc

running_loss:6114.667853593826, running_corrects:342
running_loss:6125.583394289017, running_corrects:343
running_loss:6138.1721432209015, running_corrects:344
running_loss:6155.393723726273, running_corrects:344
running_loss:6169.519154787064, running_corrects:345
running_loss:6187.382186174393, running_corrects:346
running_loss:6204.5499432086945, running_corrects:347
running_loss:6220.690057039261, running_corrects:348
running_loss:6227.757119417191, running_corrects:350
running_loss:6237.514901399612, running_corrects:352
running_loss:6251.045524835587, running_corrects:352
running_loss:6258.8230011463165, running_corrects:354
running_loss:6266.931530237198, running_corrects:356
running_loss:6281.611149072647, running_corrects:356
running_loss:6289.864547014236, running_corrects:358
running_loss:6301.057143449783, running_corrects:358
running_loss:6313.233865022659, running_corrects:359
running_loss:6327.362368822098, running_corrects:359
running_loss:6336.178597688675, running_cor

running_loss:7916.499393224716, running_corrects:511
running_loss:7928.083711385727, running_corrects:512
running_loss:7939.418037176132, running_corrects:514
running_loss:7946.136792421341, running_corrects:515
running_loss:7956.4808514118195, running_corrects:516
running_loss:7967.3078491687775, running_corrects:518
running_loss:7976.194888353348, running_corrects:519
running_loss:7983.420846700668, running_corrects:521
running_loss:7994.668908834457, running_corrects:523
running_loss:8007.9213082790375, running_corrects:524
running_loss:8021.434888601303, running_corrects:524
running_loss:8031.632726430893, running_corrects:525
running_loss:8038.534539461136, running_corrects:527
running_loss:8055.728640794754, running_corrects:527
running_loss:8062.74901509285, running_corrects:529
running_loss:8078.18380856514, running_corrects:529
running_loss:8086.657285928726, running_corrects:531
running_loss:8098.147299051285, running_corrects:531
running_loss:8105.833183526993, running_corre

...

running_loss:12672.27655673027, running_corrects:1914
running_loss:12675.281544446945, running_corrects:1918
running_loss:12684.28968501091, running_corrects:1918
running_loss:12702.359637022018, running_corrects:1919
running_loss:12708.655126333237, running_corrects:1921
running_loss:12717.882481336594, running_corrects:1923
running_loss:12727.235614538193, running_corrects:1924
running_loss:12732.151110887527, running_corrects:1927
running_loss:12744.018185853958, running_corrects:1930
running_loss:12753.29642701149, running_corrects:1931
running_loss:12761.645918130875, running_corrects:1933
running_loss:12773.008280038834, running_corrects:1934
running_loss:12782.41373372078, running_corrects:1936
running_loss:12794.978402376175, running_corrects:1937
running_loss:12801.524685621262, running_corrects:1939
running_loss:12804.143823862076, running_corrects:1942
running_loss:12814.419874429703, running_corrects:1944
running_loss:12822.393751859665, running_corrects:1946
running_loss:1

running_loss:14021.364663600922, running_corrects:2150
running_loss:14027.319501876831, running_corrects:2152
running_loss:14031.549941062927, running_corrects:2155
running_loss:14040.24095916748, running_corrects:2157
running_loss:14047.419682979584, running_corrects:2159
running_loss:14054.53285074234, running_corrects:2161
running_loss:14068.528230190277, running_corrects:2162
running_loss:14074.213902950287, running_corrects:2165
running_loss:14084.982377529144, running_corrects:2168
running_loss:14092.137101650238, running_corrects:2170
running_loss:14101.099912166595, running_corrects:2171
running_loss:14110.818129062653, running_corrects:2173
running_loss:14122.085485935211, running_corrects:2174
running_loss:14140.342215061188, running_corrects:2176
running_loss:14148.009175777435, running_corrects:2177
running_loss:14171.316894054413, running_corrects:2177
running_loss:14181.737290859222, running_corrects:2178
running_loss:14194.925080776215, running_corrects:2179
running_loss

running_loss:15389.73298573494, running_corrects:2392
running_loss:15395.41491150856, running_corrects:2394
running_loss:15409.11099076271, running_corrects:2395
running_loss:15416.574706792831, running_corrects:2398
running_loss:15422.87757563591, running_corrects:2400
running_loss:15438.555805921555, running_corrects:2400
running_loss:15451.617874860764, running_corrects:2400
running_loss:15458.700354337692, running_corrects:2402
running_loss:15469.314911603928, running_corrects:2403
running_loss:15479.744522809982, running_corrects:2405
running_loss:15489.096733808517, running_corrects:2407
running_loss:15493.199590921402, running_corrects:2410
running_loss:15496.489550590515, running_corrects:2414
running_loss:15503.089721679688, running_corrects:2416
running_loss:15508.675355911255, running_corrects:2418
running_loss:15517.497548103333, running_corrects:2419
running_loss:15523.90095615387, running_corrects:2422
running_loss:15530.54125738144, running_corrects:2423
running_loss:155

running_loss:16797.150419712067, running_corrects:2624
running_loss:16801.581821918488, running_corrects:2626
running_loss:16810.965450763702, running_corrects:2627
running_loss:16828.12445116043, running_corrects:2627
running_loss:16838.08168077469, running_corrects:2628
running_loss:16846.147140979767, running_corrects:2630
running_loss:16858.33643770218, running_corrects:2631
running_loss:16868.938915729523, running_corrects:2632
running_loss:16875.392176151276, running_corrects:2633
running_loss:16879.666936397552, running_corrects:2636
running_loss:16885.960651397705, running_corrects:2639
running_loss:16891.80635356903, running_corrects:2642
running_loss:16905.33131122589, running_corrects:2642
running_loss:16919.04124069214, running_corrects:2642
running_loss:16929.35993862152, running_corrects:2643
running_loss:16937.736042022705, running_corrects:2645
running_loss:16944.35369682312, running_corrects:2646
running_loss:16961.200803756714, running_corrects:2646
running_loss:16967

running_loss:18097.141209363937, running_corrects:2874
running_loss:18105.454817533493, running_corrects:2875
running_loss:18114.747950315475, running_corrects:2877
running_loss:18127.0270049572, running_corrects:2877
running_loss:18141.740780591965, running_corrects:2878
